<a href="https://colab.research.google.com/github/yasego/HandsOnML/blob/main/Ch3_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.  MNIST 데이터 셋으로 분류기 만들어서 테스트 세트에서 97% 정확도 달성하기
KnighborsClassifier 사용하고 weights와 n_neighbors 하이퍼 파라미터로 그리드 탐색 시도

In [ ]:
from sklearn.datasets import fetch_openml
mnist =fetch_openml('mnist_784',version=1, as_frame=False)

In [ ]:
import numpy as np

In [ ]:
X,y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)
X_train, X_test, y_train, y_test = X[:60000], X[60000:],y[:60000],y[60000:] 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ....n_neighbors=3, weights=uniform;, score=0.972 total time=  32.7s
[CV 2/5] END ....n_neighbors=3, weights=uniform;, score=0.971 total time=  32.0s
[CV 3/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  34.5s
[CV 4/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  33.4s
[CV 5/5] END ....n_neighbors=3, weights=uniform;, score=0.970 total time=  35.2s
[CV 1/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  33.4s
[CV 2/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  33.0s
[CV 3/5] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  32.3s
[CV 4/5] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  32.3s
[CV 5/5] END ...n_neighbors=3, weights=distance;, score=0.971 total time=  33.7s
[CV 1/5] END ....n_neighbors=4, weights=uniform;, score=0.969 total time=  36.0s
[CV 2/5] END ....n_neighbors=4, weights=uniform;,

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5],
                          'weights': ['uniform', 'distance']}],
             verbose=3)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

0.9714

2. MNIST 이미지를 왼, 오른, 위, 아래, 어느 방향으로든 1픽셀씩 이동시키는 함수 작성. 훈련세트의 4개의 복사본으르 만들어 훈련세트에 추가. 테이터 증식 또는 훈련세트 확장

In [ ]:
from scipy.ndimage.interpolation import shift
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])   #1차원 배열로 바꾸기 위해서 reshape([-1]) 옵션을 줌

In [ ]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

In [ ]:
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [ ]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [ ]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)

y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9763